<a href="https://colab.research.google.com/github/ELehmann91/NLP-Contract-Analysis/blob/master/EDA_with_innolytiq_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
import json

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = '/content/gdrive/Shared drives/FS-AI Base Data/innolytiq_pdf/'
!ls '/content/gdrive/Shared drives/FS-AI Base Data/innolytiq_pdf/xs1191881306.pdf/data.json'

'/content/gdrive/Shared drives/FS-AI Base Data/innolytiq_pdf/xs1191881306.pdf/data.json'


In [0]:
file_list = os.listdir(path)
len(file_list)

76

In [0]:
print(path + file_list[0]+'/data.json')

/content/gdrive/Shared drives/FS-AI Base Data/innolytiq_pdf/erfurter_bank_eg.pdf/data.json


# Example File


In [0]:
with open(path + file_list[0]+'/data.json', encoding="utf8") as json_file:
    data = json.load(json_file)
    for line in data['output'][:3]:
        print(line)
        print()

{'id': '5d9a6f1faaa2b70012237ccd', 'name': '006_issue_date', 'data': [{'top': '32.1883%', 'left': '61.1317%', 'width': '9.27644%', 'height': '1.573982939632546%', 'value': '22.02.2013', 'page': 1}]}

{'id': '5d9a6ff4aaa2b70012237ce1', 'name': '009_nominal_amount', 'description': 'EUR nominal of the issue', 'data': [{'top': '25.5151%', 'left': '35.9926%', 'width': '10.383858998144714%', 'height': '2.164534120734908%', 'value': '5.000.000,00', 'page': 1}]}

{'id': '5d9a6fc1aaa2b70012237cd4', 'name': '008_maturity_date', 'data': [{'top': '37.6115%', 'left': '48.1447%', 'width': '8.806818181818182%', 'height': '2.098917322834646%', 'value': '22.02.2022', 'page': 1}]}



# Create list of fields for Dataframe

In [0]:
name_list = []

with open(path + file_list[0]+'/data.json', encoding="utf8") as json_file:
    data = json.load(json_file)
    for line in data['output']:
        try: #line['members'] is not None:
            name_list.append(line['name'])
        except:
            print(line)

In [0]:
df_ = pd.DataFrame(index=file_list, columns=name_list)
df_ = df_.fillna(0) # with 0s rather than NaNs

## Create Dataframe

In [0]:
for file in file_list:
    with open(path+file+'/data.json', encoding="utf8") as json_file:
        data = json.load(json_file)
        #print(data)
        try:
            for line in data['output']:
                try: # line['members'] is not None:
                    df_.loc[df_.index==file,line['name']] = line['data'][0]['value']
                except:
                    pass
                    #print(line['name'])
        except KeyError:
            print(file)

In [0]:
df_

,006_issue_date,009_nominal_amount,008_maturity_date,010_redemption_formula,011_seniority,012_business_day_convention,014_business_day_calendar,017_first_payment_date,018_payment_dates,021_formula,002_issuer_name,020_paragraph_489,004_identifiers,007_issue_price,003_product_subclass,015_payment_frequency,005_currency,013_day_count_convention,001_type,004_wkn,004_isin,023_amendment,022_legislation
erfurter_bank_eg.pdf,22.02.2013,"5.000.000,00",22.02.2022,Nennwert,,,,22.02.2014,22.02.,"2,34 Prozent",Die Erfurter Bank eG,,0,"100,00%",Schuldschein,1,EUR,act/act,Fixed,,,no_amendment,Ger
de000dr0q3k7_en.pdf,23 July 2008,"100,000,000",21 September 2012,100%,,,,21 September 2009,21 September 2009 \n21 September 2010 \n21 Sep...,10 years EURO - Swap minus 2 years EURO Swap a...,"Dresdner Bank Aktiengesellschaft, Frankfurt am...",,0,100,other Loan,1,EUR,,Floating,DR0Q3K,DE 000 DR0 Q3K 7,no_amendment,Ger
xf0000056977_-_fresenius_se.pdf,"January 31, 2017",246000000,"January 31, 2024",,,,TARGET2,"January 31, 2018",,,Fresenius SE & Co. KGa,Excluded,0,,Schuldschein,1,EUR,Act/act ICMA,Fixed,,,no_amendment,Ger
xf7000001779.pdf,22.06.2015,10.000.000,22.06.2023,,,,,22.06.2016,22.06.,"1,00%",Münchener Hypothekenbank eG,,0,,Schuldschein,1,,Act/act ICMA,Fixed,,XF7000001779,no_amendment,
xf0000053353_-_bayern_lb_nasv.pdf,,"2.000.000,--",11.September.2023,zu pari,,,,11.September. 2009,11.September,"5,40 %","Bayerische Landesbank, München",,0,,Schuldschein,1,EUR,Act/act ICMA,Fixed,,XF7000NP3155,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
de000cz225q7_dt.pdf,16. August 2010,50.000.000,16. August 2016,Nennwert,Unsecured,Following unadjusted,TARGET,16. August 2011,16. August,3-Monats EURIBOR,Commerzbank Aktiengesellschaft,Not Excluded,0,100,Bond,1,EUR,act/act,Floating,CZ2 25Q,DE000CZ225Q7,no_amendment,Ger
de000cb83gn5_dt.pdf,25.05.2011,"5.000.000,00",25.05.2016,"100,00%",Senior Unsecured,,,,25.05.,"Jahr 1 bis Jahr 2: \n3,10% p.a.\nJahr 3 bis 5:...",Commerzbank AG,Not Excluded,0,"100,00",Bond,1,EUR,,Floating,CB83GN,DE000CB83GN5,no_amendment,Ger
xf7000001873.pdf,,"2.000.000,00",04.27.2026,,,,,,27.04.,"1,50 %",Sudwestbank AG,,0,,Schuldschein,1,EUR,,Fixed,,XF7000001873,no_amendment,
de000dr0q3k7_dt.pdf,23. Juli 2008,"100.000.000,0",21. September 2012,100%,,,,21. September2009,21. September2009\n21. September2010\n21. Sept...,"Pramie B,35% des Nominals pro Laufzeitjahr, fa...",Dresdner Bank Aktiengesellschaft,Not Excluded,0,100,other Loan,,EUR,,Floating,DR0Q3K,DE 000 DR0Q3K7,no_amendment,Ger


### EDA

In [0]:
for col in df_.columns: 
    print(col) 

006_issue_date
009_nominal_amount
008_maturity_date
010_redemption_formula
011_seniority
012_business_day_convention
014_business_day_calendar
017_first_payment_date
018_payment_dates
021_formula
002_issuer_name
020_paragraph_489
004_identifiers
007_issue_price
003_product_subclass
015_payment_frequency
005_currency
013_day_count_convention
001_type
004_wkn
004_isin
023_amendment
022_legislation


In [0]:
for col in df_.columns:
  if len(df_[col].unique()) > 20:
    pass
  else:
    print(col)
    print(df_[col].value_counts())
    print('\n')

011_seniority
                    52
Senior Secured      12
Senior Unsecured     9
Unsecured            2
Subordinated         1
Name: 011_seniority, dtype: int64


012_business_day_convention
                                 40
Following unadjusted             24
Modified following adjusted       7
Modified following unadjusted     2
Following adjusted                2
Modified Following                1
Name: 012_business_day_convention, dtype: int64


014_business_day_calendar
                                36
TARGET                          19
TARGET2                         16
TARGET2,                         1
London and TARGET                1
New York, London and TARGET2     1
TARGET & Sydney                  1
London and TARGET2               1
Name: 014_business_day_calendar, dtype: int64


020_paragraph_489
                              42
Excluded                      25
Not Excluded                   8
10 Jahren gemäß § 489 Abs.     1
Name: 020_paragraph_489, dtype: int64

Code to find PDF's that need to be Redone

In [0]:
print(df_.index[df_['022_legislation'] == 'Netherlands'])
df_.index[df_['022_legislation'] == 'NLD']

Index(['amba-#7711630-v1-executed_-_nwb_-_sdg_final_terms.pdf',
       'final_terms_eur250m_series_1516.pdf',
       'final_terms_series_1411_tranche_6.pdf',
       'icm-#33492791-v1-executed_final_terms_-_nwb_aug_2021.pdf',
       'series_1513_x_final_terms.pdf'],
      dtype='object')


Index(['final_terms_gbp15m_series_1510.pdf',
       'final_terms_xs2071397348_series_1523.pdf',
       'final_terms_series_1515.pdf',
       'final_terms_series_1418_ii_xs1312042648.pdf',
       'final_terms_usd100m_series_1513_vi.pdf'],
      dtype='object')